In [17]:
#import networkx as nx
import csv
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib;
import pandas as pd 
import utm
import matplotlib.pyplot as plt 
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *
from ipyleaflet import Map, FullScreenControl, Polyline
from datetime import datetime, timedelta, timezone
import subprocess
import numpy as np
ox.config(use_cache=True, log_console=True)
%matplotlib inline
from shapely.geometry import Point
import pyproj, math

CLIGHT = 299792458.0 * 1e-9
class GpsObs:

    def __init__(self, data):
         # Class attributes
        self.time = data[1]
        self.name = data[2]
        self.satNum = data[3]
        self.carrier1 = data[5] # carrier-phase cycle 1
        self.carrier2 = data[6] # carrier-phase cycle 2
        self.prng1 = data[7] # pseudorange freq 1
        self.prng2 = data[8] # pesudorange freq 2
        self.doppler1 = data[9] # doppler frequency [Hz]
        self.doppler2 = data[10] # doppler frequency [Hz]
        self.satPos = [0, 0, 0]
        self.dts = 0
       # self.occupGrid = OccupancyGrid()            # Generated Occupancy Grid  

gps_trace_file_path = "../dataset/UrbanNav/Tokyo/Odaiba/results.pos.trace"


# Table of dictionaries containing multiple obs_set
obs_tab = []
prng_set = {}
with open(gps_trace_file_path, 'r') as f:
    reader = csv.reader(f, delimiter='\n')
    data_trace = list(reader)
    idx_obs_s = -1
    idx_obs_e = -1
    idx_nav_s = -1
    idx_nav_e = -1
    # Set of multiple GPS observations (per given time)
    obs_set = {}

    for i, line in enumerate(data_trace[:1000]):
        # Convert array to string
        line = line[0]
        # Split everything in the same way
        line = line.split('/')[-1]
        if '#OBSERVATIONS_START' in line:
            idx_obs_s = i
        if '#OBSERVATIONS_END' in line:
            idx_obs_e = i
            # Add obs_set to the table and reset its value
            obs_tab.append(obs_set)
            obs_set={}
        if '#SATPOSITION_START' in line:
            idx_nav_s = i
        if '#SATPOSITION_END' in line:
            idx_nav_e = i
            
        if 'PRNG' in line:
            line2 = line.split()
            if line2[2] not in prng_set:
                prng_set[line2[2]] = [line2[3], line2[4]]
        
        if i > idx_obs_s and ((i < idx_obs_e) or (idx_obs_e < idx_obs_s)):
            line = line.split()
            if line[4] == 'rcv2':  # rcv2 - stationary base receiver
                continue
            new_obs = GpsObs(line)
            obs_set[new_obs.satNum] = new_obs
            
        if i > idx_nav_s and ((i < idx_nav_e) or (idx_nav_e < idx_nav_s)):
            line = line.split()
            satNum = line[2]
            dts = line[7]
            obs = obs_tab[-1]
            if(satNum in obs):
                obs[satNum].satPos = [float(line[3]), float(line[4]), float(line[5])]
                obs[satNum].dts = dts

for key, value in  prng_set.items():
    print (key, value)
    
out_path = '/home/kcwian/Workspace/catkin_gnss_2/src/my_g2o/build/gps.g2o'
with open(out_path, 'w') as f:

    # Initial vertex # number, pose, quaternion
    f.write('VERTEX_SE3:QUAT 0 0 0 0 0 0 0 1\n')
    v_idx = 0
    e_idx = -0
    for obs_set in obs_tab:
        for key, value in  obs_set.items():
            v_idx += 1
            if(key in prng_set):
                if (float(prng_set[key][0]) > 0):
                  #if (float(value.prng2) != 0.0):
                    f.write('VERTEX_SE3:QUAT {} {} {} {} 0 0 0 1\n'.format(v_idx, value.satPos[0], value.satPos[1], value.satPos[2]))
                    f.write('FIX {}\n'.format(v_idx))
                    f.write('GPSEDGE 0 {} {} {}\n'.format(v_idx, prng_set[key][0], 1.0/math.sqrt(float(prng_set[key][1]))))
                    #f.write('GPSEDGE 0 {} {} {}\n'.format(v_idx, str(((float(value.prng2) - 1.6469 * float(value.prng1))/(1-1.6469)) + float(value.dts) * CLIGHT  + 99240) , 1.0/math.sqrt(float(prng_set[key][1]))))
                    print("RNG1 {}, RNG2 {}, comb: {}".format(value.prng1, value.prng2, ((float(value.prng2) - 1.6469 * float(value.prng1))/(1-1.6469))))
        for key, value in  prng_set.items():
            e_idx += 1
            # Edge from, to, distance, noise
            noise = 1000
            # f.write('GPSEDGE 0 {} {} {}\n'.format(e_idx, value, 1.0/noise))

        for key, value in  obs_set.items():
            pass
        
        # Only first for test
        break;
        
        
g2o_result_path = '/home/kcwian/Workspace/catkin_gnss_2/src/my_g2o/build/g2o_result.txt'
subprocess.call("./MyGPS", shell=True, cwd="/home/kcwian/Workspace/catkin_gnss_2/src/my_g2o/build/")
with open(g2o_result_path, 'r') as f:
    res = f.readline().split()
    print(res)
    

ecef_gpd = gpd.points_from_xy([res[0]], [res[1]], z=[res[2]], crs="EPSG:4978")
lon_lat_gpd = ecef_gpd.to_crs(crs="EPSG:4326")
print(lon_lat_gpd[0].y, lon_lat_gpd[0].x, lon_lat_gpd[0].z )

gps_result_file_path = "../dataset/UrbanNav/Tokyo/Odaiba/results.pos"
point = [lon_lat_gpd[0].y, lon_lat_gpd[0].x]
m = Map(center=point, basemap=basemaps.OpenStreetMap.Mapnik, zoom=15)
marker = Marker(location=point, draggable=True)
rtklib_est_ecef = [-3963425.455 ,3350879.447 ,3694864.328]
rtklib_est_ecef_gpd = gpd.points_from_xy([rtklib_est_ecef[0]], [rtklib_est_ecef[1]], z=[rtklib_est_ecef[2]], crs="EPSG:4978")
rtklib_est_lonlat = rtklib_est_ecef_gpd.to_crs(crs="EPSG:4326")
marker2 = Marker(location=(rtklib_est_lonlat[0].y, rtklib_est_lonlat[0].x), draggable=True)
tmp = [-3963430.81947 ,  3350878.14087  , 3694869.42872]
markerNew = Marker(location=(tmp[0], tmp[1]), draggable=True)
m.add_layer(marker)
m.add_layer(marker2)

print("Ref vs Est:", ecef_gpd[0].x - rtklib_est_ecef_gpd[0].x,ecef_gpd[0].y - rtklib_est_ecef_gpd[0].y,ecef_gpd[0].z - rtklib_est_ecef_gpd[0].z )
print("Ref vs Est2:", tmp[0] - rtklib_est_ecef_gpd[0].x,tmp[1] - rtklib_est_ecef_gpd[0].y,tmp[2] - rtklib_est_ecef_gpd[0].z )


m

5 ['26353560.911', '31.438']
15 ['23339047.148', '31.117']
20 ['25343445.220', '31.360']
21 ['25664032.015', '31.945']
24 ['23119120.426', '31.223']
38 ['24409406.464', '49.907']
39 ['22704302.635', '49.672']
40 ['24863202.972', '50.011']
41 ['22371664.918', '49.817']
48 ['25430170.606', '49.823']
60 ['27400308.310', '34.115']
63 ['26760359.925', '34.109']
68 ['27514679.320', '34.241']
78 ['30511421.566', '34.302']
90 ['27542560.777', '34.325']
96 ['41213932.559', '36.955']
97 ['39981340.104', '37.067']
109 ['40493028.637', '31.782']
111 ['39757069.712', '31.643']
114 ['41261458.367', '32.006']
119 ['24953722.332', '31.588']
121 ['39841152.129', '31.682']
108 ['37575017.883', '31.715']
113 ['38525837.854', '31.924']
RNG1 23338470.203, RNG2 23338476.309, comb: 23338460.764137737
RNG1 20424739.578, RNG2 0.000, comb: 51997995.99784851
RNG1 22172560.578, RNG2 22172565.426, comb: 22172553.083796885
RNG1 22734445.977, RNG2 0.000, comb: 57878125.02631211
RNG1 20122061.133, RNG2 20122068.316, 

Map(center=[35.62937098495249, 139.7870604143194], controls=(ZoomControl(options=['position', 'zoom_in_text', …